In [1]:
import time
import nrrd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import h5py

from dipy.tracking import eudx
from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel, quantize_evecs
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model, DeterministicMaximumDirectionGetter
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core import gradients
from dipy.tracking.streamline import Streamlines, transform_streamlines
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, gradient_table_from_bvals_bvecs
from dipy.reconst.dti import fractional_anisotropy
from dipy.tracking import utils

import src.dwi_tools as dwi_tools
import src.nn_helper as nn_helper

Using TensorFlow backend.


In [ ]:
pTrainData = 'data/train_wholebrain_dti_step01.h5'
#pTrainInput = 'train_input_normalized_dti_cs1_wholebrain_'
noCrossings = 3

# Preprocessing

crop multi-shell DWI to single shell data

In [2]:
bvals,bvecs,gtab,dwi,aff,t1,binarymask = dwi_tools.loadHCPData('data/100307')
dwi_subset, gtab_subset, bvals_subset, bvecs_subset = dwi_tools.cropDatsetToBValue(1000, bvals, bvecs, dwi)

The average b0 image is required for data normalization.

In [3]:
b0_idx = bvals < 10
b0 = dwi[..., b0_idx].mean(axis=3)

In [4]:
dwi_singleShell = np.concatenate((dwi_subset, dwi[..., b0_idx]), axis=3)
bvals_singleShell = np.concatenate((bvals_subset, bvals[..., b0_idx]), axis=0)
bvecs_singleShell = np.concatenate((bvecs_subset, bvecs[b0_idx,]), axis=0)
gtab_singleShell = gradient_table(bvals=bvals_singleShell, bvecs=bvecs_singleShell, b0_threshold = 10)
dwi_singleShell_norm = dwi_tools.normalize_dwi(dwi_singleShell, b0)

Percentage erroneous voxels: 4.07


/home/nico/Code/deepFibreTracking/src/dwi_tools.py:106: RuntimeWarning: divide by zero encountered in true_divide
  weights_normed = (weights / b0)
/home/nico/Code/deepFibreTracking/src/dwi_tools.py:106: RuntimeWarning: invalid value encountered in true_divide
  weights_normed = (weights / b0)


project data into spherical harmonics

# Tractography

In this section we'll generate streamlines using different approaches. This is gonna be the foundation for the evaluation of our method. We'll also employ simulated as well as curated data for our analysis.

In [13]:
step_size = 0.6 # mm

In [14]:
# roi_idx = (slice(20, 50), slice(55, 85), slice(38, 39)) #  splenium of the corpus callosum
from dipy.tracking.utils import random_seeds_from_mask, seeds_from_mask
ccmask, options = nrrd.read('data/100307/100307-ccSegmentation.nrrd')
ccseeds = seeds_from_mask(ccmask, affine=aff)
wholebrainseeds = seeds_from_mask(binarymask, affine=aff)
validationSeeds = ccseeds[45:48]
rndseeds = random_seeds_from_mask(binarymask, seeds_count=4000, seed_count_per_voxel=False, affine=aff)

## Single Tensor Model

In [ ]:
import dipy.reconst.dti as dti
start_time = time.time()
dti_model = dti.TensorModel(gtab_singleShell)
dti_fit = dti_model.fit(dwi_singleShell_norm, mask=binarymask)
dti_fit_odf = dti_fit.odf(sphere = default_sphere)
dg = DeterministicMaximumDirectionGetter
dg = dg.from_pmf(dti_fit_odf, max_angle=30., sphere=default_sphere)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + 's')

In [ ]:
dtiPeakDirs = dtipeaks.peak_dirs[:,:,:,0,:]

spherical harmonics

In [ ]:
from dipy.data import default_sphere
from dipy.direction import DeterministicMaximumDirectionGetter

start_time = time.time()

csd_model = ConstrainedSphericalDeconvModel(gtab_singleShell, None, sh_order=6)
csd_fit = csd_model.fit(dwi_singleShell_norm, mask=binarymask)


dg = DeterministicMaximumDirectionGetter.from_shcoeff(csd_fit.shm_coeff,
                                                             max_angle=30.,
                                                             sphere=default_sphere)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + 's')

track and visualize streamlines

In [ ]:
from dipy.tracking.local import BinaryTissueClassifier
classifier = ThresholdTissueClassifier(dti_fit.fa, .15)
streamline_generator = LocalTracking(dg, classifier, wholebrainseeds, aff, step_size=step_size)
streamlines = list(streamline_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 40)

In [ ]:
import importlib
importlib.reload(dwi_tools)
import src.dwi_tools as dwi_tools
from dipy.tracking.streamline import transform_streamlines
streamlines_imageCS = transform_streamlines(sl_r,np.linalg.inv(aff)) # project streamlines from RAS into image (voxel) coordinate system
dwi_tools.visStreamlines(streamlines_imageCS,t1)

In [ ]:
dwi_tools.saveVTKstreamlines(sl_r,'baseline_dti_sw1.vtk')

In [ ]:
classifier = ThresholdTissueClassifier(dti_fit.fa, .2) # used to be 0.01 10/16/18
streamline_generator = LocalTracking(dg, classifier, validationSeeds, aff, step_size=.1)
streamlines_val = list(streamline_generator)
streamlines_val_filtered = dwi_tools.filterStreamlinesByLength(streamlines_val, 40)
streamlines_val_imageCS = transform_streamlines(streamlines_val_filtered,np.linalg.inv(aff)) # project streamlines from RAS into image (voxel) coordinate system
np.save('streamlines_validation.npy', streamlines_val_imageCS)

## Q-ball Reconstruction

In [ ]:
csamodel = CsaOdfModel(gtab, 4)
sphere = get_sphere('symmetric724')
start_time = time.time()
csapeaks = peaks_from_model(model=csamodel,
                            data=dwi,
                            sphere=sphere,
                            relative_peak_threshold=.5,
                            min_separation_angle=25,
                            mask=binarymask,
                            return_odf=False,
                            parallel=True,
                            normalize_peaks=False)

GFA = csapeaks.gfa
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s / GFA.shape (%d, %d, %d)' % GFA.shape)

tracking

In [ ]:
classifier = ThresholdTissueClassifier(dtipeaks.gfa, .25)
streamlines_generator = LocalTracking(csapeaks, classifier, seeds, np.identity(4), step_size=.5)
streamlines = Streamlines(streamlines_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 50)

## Constrained Spherical Deconvolution

Use in case of b = 2,500 – 3,000 s/mm² data

In [5]:
response, ratio = auto_response(gtab_singleShell, dwi_singleShell, roi_radius=10, fa_thr=0.7)

In [8]:
csd_model = ConstrainedSphericalDeconvModel(gtab_singleShell, response)
sphere = get_sphere('symmetric724')
start_time = time.time()
csd_peaks = peaks_from_model(model=csd_model,
                             data=dwi_singleShell,
                             sphere=sphere,
                             mask=binarymask,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             parallel=True)
GFA = csd_peaks.gfa
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s / GFA.shape (%d, %d, %d)' % GFA.shape)

Runtime 558.4986169338226 s / GFA.shape (145, 174, 145)


In [9]:
import dipy.reconst.dti as dti
start_time = time.time()
dti_model = dti.TensorModel(gtab_singleShell)
dti_fit = dti_model.fit(dwi_singleShell_norm, mask=binarymask)
fa = dti_fit.fa
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + 's')

Runtime 91.96015524864197s


In [18]:
start_time = time.time()
classifier = ThresholdTissueClassifier(dti_fit.fa, .1)
streamlines_generator = LocalTracking(csd_peaks, classifier, wholebrainseeds, aff, step_size=step_size)
streamlines = Streamlines(streamlines_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 50)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + 's')

Runtime 267.20205879211426s


In [19]:
dwi_tools.saveVTKstreamlines(streamlines_filtered,'baseline_csd_sw06_.vtk')

0/571885
1000/571885
2000/571885
3000/571885
4000/571885
5000/571885
6000/571885
7000/571885
8000/571885
9000/571885
10000/571885
11000/571885
12000/571885
13000/571885
14000/571885
15000/571885
16000/571885
17000/571885
18000/571885
19000/571885
20000/571885
21000/571885
22000/571885
23000/571885
24000/571885
25000/571885
26000/571885
27000/571885
28000/571885
29000/571885
30000/571885
31000/571885
32000/571885
33000/571885
34000/571885
35000/571885
36000/571885
37000/571885
38000/571885
39000/571885
40000/571885
41000/571885
42000/571885
43000/571885
44000/571885
45000/571885
46000/571885
47000/571885
48000/571885
49000/571885
50000/571885
51000/571885
52000/571885
53000/571885
54000/571885
55000/571885
56000/571885
57000/571885
58000/571885
59000/571885
60000/571885
61000/571885
62000/571885
63000/571885
64000/571885
65000/571885
66000/571885
67000/571885
68000/571885
69000/571885
70000/571885
71000/571885
72000/571885
73000/571885
74000/571885
75000/571885
76000/571885
77000/571885

### Store intermediate data

In [ ]:
np.save(pTrainInput + "_sl_filt.npy",streamlines_filtered)
np.save(pTrainInput + "_seeds.npy",ccseeds)

# Generate Training data

In [22]:
noX = 1
noY = 1
noZ = 1
coordinateScaling = 1

In [20]:
sl_r = np.random.choice(streamlines_filtered,30000, replace=False)

In [23]:
import importlib
importlib.reload(dwi_tools)
import src.dwi_tools as dwi_tools

#rawData = dwi_B0normalized
#rawData = dtiPeakDirs
#rawData = dti_fit.quadratic_form.reshape([145,174,145,9])
#rawData = dti_fit_odf
#rawData = csd_fit.shm_coeff # spherical harmonics
rawData = data_sh
#rawData = tensors
#rawData = dtiPeakDirs

start_time = time.time()
train_DWI,train_prevDirection, train_nextDirection = dwi_tools.generateTrainingData(sl_r, rawData, affine=aff, noX=noX,noY=noY,noZ=noZ,coordinateScaling=coordinateScaling,distToNeighbours=1, noCrossings = noCrossings, step = 1)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

NameError: name 'data_sh' is not defined

In [ ]:
train_DWI = train_DWI[0:2058818,]

In [ ]:
train_prevDirection = train_prevDirection[0:2058818,]
train_nextDirection = train_nextDirection[0:2058818,]

In [ ]:
train_nextDirection[-1000,]

In [ ]:
pTrainData_fibrePrediction = 'train_OLDsh4_step%.1f_wholeBrain_b1k_csd_%dx%dx%d.h5' % (step_size,noX,noY,noZ)

In [ ]:
with h5py.File(pTrainData_fibrePrediction,"w") as f:
    f.create_dataset('train_DWI',data=train_DWI)
    f.create_dataset('train_curPosition',data=train_prevDirection)   
#   f.create_dataset('train_LikelyFibreDirections',data=train_LikelyFibreDirections)   
    f.create_dataset('train_NextFibreDirection',data=train_nextDirection)   

In [ ]:
sl_r_imageCS = transform_streamlines(sl_r,np.linalg.inv(aff)) # project streamlines from RAS into image (voxel) coordinate system
dwi_tools.visStreamlines(sl_r_imageCS,t1)